# CLASSIFICADOR DE REVIEWS

## DADOS 

In [ ]:
#https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/


In [1]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import random
from random import shuffle
from nltk.stem.snowball import SnowballStemmer
import string
import re
from nltk.corpus import stopwords


In [2]:
##carregando dados, filtrando-os e salvando e um novo objeto

dados = pd.read_csv('data/reviewFinal.csv',encoding='latin-1')
corpus = pd.DataFrame()
listTitle = []
listRecom = []
prediTitle = []

for i, r in dados.iterrows():
    if r['recommend'] != 'NI' and r['title'] != "" and r['title'] is not np.nan: # and r['site'] == "glassdoor":
        listTitle.append(r['title'])
        listRecom.append(r['recommend'])
    elif r['recommend'] == 'NI' and r['title'] is not np.nan:
        prediTitle.append(r['title'])

## criando objeto para encoder
encoder = LabelEncoder()

In [3]:
corpus['title']     = listTitle
corpus['recommend'] = listRecom

In [38]:
### GERANDO LINHAS PARA DADOS DE TREINO E TESTE
x = [ i for i in range(len(corpus))]
shuffle(x)

rows_train =  x[0:round( len(corpus) * .80)] #25000
rows_test  = x[round( len(corpus) * .80):(len(corpus))]
dataset = []

In [5]:
### LIMPANDO TEXTO
stemmerEng  = SnowballStemmer("english")
stemmerPort = SnowballStemmer("portuguese")

sClear = []
tr = str.maketrans("", "", string.punctuation)
i = 0
for i,text in corpus.iterrows():    
    #print(text['title'].split())
    clear = ""
    for word in text['title'].split():        
        word = word.lower() ## CONVERTENDO PARA MINUSCULO
        word = word.translate(tr) ##REMOVENDO PONTUAÇÕES        
        word = re.sub("\d"," ",word) ## REMOVENDO NUMEROS
        word = stemmerEng.stem(word) ## STEMM
        word = stemmerPort.stem(word) ## STEMM
        
        clear = clear + ' ' + word
        clear = re.sub("\s+"," ",clear) ## REMOVENDO ESPAÇOS DUPLICADOS
        clear = clear.strip()

        
    sClear.append(clear)
    text['title'] = clear
    i = i + 1


In [6]:
prediTitleClean = []
for title in prediTitle:
    clear = ""
    for word in title.split():        
        word = word.lower() ## CONVERTENDO PARA MINUSCULO
        word = word.translate(tr) ##REMOVENDO PONTUAÇÕES        
        word = re.sub("\d"," ",word) ## REMOVENDO NUMEROS
        word = stemmerEng.stem(word) ## STEMM
        word = stemmerPort.stem(word) ## STEMM
        
        clear = clear + ' ' + word
        clear = re.sub("\s+"," ",clear) ## REMOVENDO ESPAÇOS DUPLICADOS
        clear = clear.strip()
        
    prediTitleClean.append(clear)

In [39]:
train   = corpus.iloc[rows_train, [0]]
train   = train['title'].tolist()
train_y = corpus.iloc[rows_train, 1 ] 
train_y = encoder.fit_transform(train_y)

test   = corpus.iloc[rows_test, [0]]
test   = test['title'].tolist()
test_y = corpus.iloc[rows_test, 1]
test_y = encoder.fit_transform(test_y)

print(set(train_y), set(test_y) )

{0, 1} {0, 1}


In [9]:
print(train_y)
print(corpus.iloc[rows_train, [1] ] )

[1 1 1 ..., 1 1 1]
      recommend
46789       Yes
28898       Yes
61256       Yes
37821       Yes
7345        Yes
28375        No
84198       Yes
42928       Yes
55422       Yes
22033        No
2696         No
74120        No
62215       Yes
85255       Yes
3253         No
44708        No
69384       Yes
70937       Yes
7468        Yes
62814       Yes
72456       Yes
39490       Yes
62435       Yes
81926        No
13182        No
11189        No
22850       Yes
82751        No
3711        Yes
59279       Yes
...         ...
42320        No
80890       Yes
3264        Yes
12994       Yes
9080        Yes
38983        No
54759        No
38233        No
27723       Yes
29867       Yes
10008        No
45771       Yes
13471       Yes
83411       Yes
67625       Yes
30575        No
22091       Yes
46422        No
4427        Yes
58270       Yes
58315       Yes
11184       Yes
56693       Yes
11543       Yes
25194       Yes
64445       Yes
74510       Yes
136         Yes
82677       Yes
11065

# MODELOS 

### NAIVE BAYES MULTINOMIAL

In [40]:
#criando countVectorizer
stopWords = stopwords.words('english') ##STOP WORD ENGLHIS
[stopWords.append(w) for w in stopwords.words('portuguese')] ##STOP WORD PORTUGUES

##STOP WORD NOME DAS EMPRESAS
nameCompany = dados.company
#[stopWords.append(i) for i in set(nameCompany)]

count_vect     = CountVectorizer(stop_words = stopWords)
X_train_counts = count_vect.fit_transform(train)
count_vect.vocabulary_.get(u'algorithm')
X_train_counts.shape

(68870, 8731)

In [41]:
# criando TF-IDM
tfidf_transformer = TfidfTransformer()
X_train_tfidf     = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(68870, 8731)

In [48]:
#CRIANDO OBJETO NAIVE BAYES E TREINANDO O MODELO
clf = MultinomialNB().fit(X_train_tfidf, train_y)

In [43]:
# FAZENDO TRANSFORMAÇÃO NOS DADOS DE TESTE E TESTANDO O MODELO NB
X_new_counts = count_vect.transform(test)
X_new_tfidf  = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

In [44]:
## Verificando acuracia do modelo
print( confusion_matrix(test_y, predicted) ) 
print( '\n acuracia : ', accuracy_score(test_y, predicted) )

# acuracia :  
# 0.7615191287349903
# 0.763141081489
# 0.764825463205

[[ 2322  3399]
 [  650 10846]]

 acuracia :  0.764825463205


In [45]:
### salvando modelo
import pickle
f = open('NBMult.pickle', 'wb')
pickle.dump(clf, f)
f.close()

### classificando comentários sem recomendações

In [49]:
X_predict_counts = count_vect.transform(prediTitleClean)
X_predict_tfidf  = tfidf_transformer.transform(X_predict_counts)

recom_predicted = clf.predict(X_predict_tfidf)

In [50]:
## Label (yes ou no) dos valores previstos
#np.unique(recom_predicted,return_counts=True)

labelPred = [ 'Yes' if rec == 1 else 'No' for rec in recom_predicted ]

In [51]:
### salvando previsoes de recomendações
pd.DataFrame({'predRecom': labelPred,'title': prediTitle}).to_csv('data/recommendPred.csv')

### BINOMIAL NAIVE BAYES

In [52]:
nb = BernoulliNB()
nb.fit(X_train_tfidf, train_y)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [53]:
predictBIN = nb.predict(X_new_tfidf)

In [54]:
print( confusion_matrix(test_y, predicted) ) 
print( '\n acuracia : ', accuracy_score(test_y, predictBIN) )
#  acuracia :  
# 0.760236975083


[[ 2322  3399]
 [  650 10846]]

 acuracia :  0.760236975083


### SVC

In [55]:
#Criando modelo SVC com Pipeline
modeloSVC  = Pipeline([('vect', TfidfVectorizer(stop_words=stopwords.words('english'))), 
                    ('clf', SVC(kernel = 'linear', probability = True))])

In [56]:
#treinando modelo
modeloSVC.fit(train, train_y)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [ ]:
predictSVC = modeloSVC.predict(test)


In [ ]:
print( confusion_matrix(test_y, predictSVC) ) 
print( '\n acuracia : ', accuracy_score(test_y, predictSVC) )


### SVM

In [ ]:
modeloSVM = Pipeline([('vect', CountVectorizer(stopwords.words('english'))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42,
                                           max_iter=500, tol=None)),
])
modeloSVM.fit(train, train_y)  

predicted = modeloSVM.predict(test)
print( confusion_matrix(test_y, predicted) ) 
print( '\n acuracia : ', accuracy_score(test_y, predicted) )

# FIM!!